In [79]:
import pandas as pd
import os
import json


RQ1

In [80]:
xss_acc_file = "/workspaces/RAG_secure_code_generation/rq1_large.csv"
xss_f2_file = "/workspaces/RAG_secure_code_generation/rq1_large_f2.csv"
sqli_acc_file = "/workspaces/RAG_secure_code_generation/rq1_large_sqli.csv"
sqli_f2_file = "/workspaces/RAG_secure_code_generation/rq1_large_f2_sqli.csv"

In [81]:
df_rq1_xss_acc = pd.read_csv(xss_acc_file)
df_rq1_xss_f2 = pd.read_csv(xss_f2_file)
df_rq1_sqli_acc= pd.read_csv(sqli_acc_file)
df_rq1_sqli_f2 = pd.read_csv(sqli_f2_file)

df_rq1 = pd.concat([df_rq1_xss_acc,
                    df_rq1_xss_f2,
                    df_rq1_sqli_acc,
                    df_rq1_sqli_f2
                    ])

In [82]:
def get_avg_difference(df):
    #keep only the columns starting with no_rag or rag
    df = df.filter(regex='^no_rag|^rag')
    #get all the number present in the last split of _ 
    numbers = set(map(lambda x: int(x.split("_")[-1]), list(df.columns)))
    diff = 0
    for n in numbers:
        diff += df[f"rag_{n}"].sum() - df[f"no_rag_{n}"].sum()
    return diff/len(numbers)/len(df)

In [90]:
def get_max_difference(df):
    #keep only the columns starting with no_rag or rag
    df = df.filter(regex='^no_rag|^rag')
    #reset index
    df.reset_index(drop=False, inplace = True)
    #get all the number present in the last split of _ 
    numbers = set(map(lambda x: int(x.split("_")[-1]), list(df.columns)))
    diffs = []
    for i, row in df.iterrows():
        for n in numbers:
            print(df.iloc[i][f"rag_{n}"], df.iloc[i][f"no_rag_{n}"],df.iloc[i][f"rag_{n}"] - df.iloc[i][f"no_rag_{n}"])
            diffs.append(df.iloc[i][f"rag_{n}"] - df.iloc[i][f"no_rag_{n}"])
    return max(diffs)

In [99]:
df_rq1_xss_acc.head()

,model_temperature,no_rag_0,no_rag_1,no_rag_3,no_rag_5,rag_0,rag_1,rag_3,rag_5,avg_acc_diff
0,anthropic-claude-3-opus_0.0,0.767,0.787,0.953,0.873,0.803,0.786,0.897,0.926,0.008
1,anthropic-claude-3-opus_0.5,0.723,0.812,0.831,0.813,0.751,0.702,0.839,0.870,-0.004
2,anthropic-claude-3-opus_1.0,0.784,0.768,0.832,0.757,0.796,0.753,0.762,0.805,-0.006
3,anthropic-claude-3-sonnet_0.5,0.540,0.579,0.557,0.594,0.614,0.580,0.591,0.569,0.021
4,anthropic-claude-3-sonnet_1.0,0.562,0.619,0.601,0.625,0.656,0.580,0.606,0.539,-0.007


In [98]:
pd.concat([df_rq1_xss_acc, df_rq1_xss_f2]).head()

,model_temperature,no_rag_0,no_rag_1,no_rag_3,no_rag_5,rag_0,rag_1,rag_3,rag_5,avg_acc_diff,avg_f2_diff
0,anthropic-claude-3-opus_0.0,0.767,0.787,0.953,0.873,0.803,0.786,0.897,0.926,0.008,NaN
1,anthropic-claude-3-opus_0.5,0.723,0.812,0.831,0.813,0.751,0.702,0.839,0.870,-0.004,NaN
2,anthropic-claude-3-opus_1.0,0.784,0.768,0.832,0.757,0.796,0.753,0.762,0.805,-0.006,NaN
3,anthropic-claude-3-sonnet_0.5,0.540,0.579,0.557,0.594,0.614,0.580,0.591,0.569,0.021,NaN
4,anthropic-claude-3-sonnet_1.0,0.562,0.619,0.601,0.625,0.656,0.580,0.606,0.539,-0.007,NaN


In [102]:
print(f"XSS ACC diff: {get_max_difference(pd.concat([df_rq1_xss_f2]))}")


0.662 0.614 0.04800000000000004
0.663 0.621 0.04200000000000004
0.835 0.922 -0.08700000000000008
0.877 0.782 0.09499999999999997
0.587 0.502 0.08499999999999996
0.48 0.669 -0.18900000000000006
0.722 0.712 0.010000000000000009
0.778 0.672 0.10599999999999998
0.657 0.63 0.027000000000000024
0.575 0.598 -0.02300000000000002
0.583 0.712 -0.129
0.657 0.592 0.06500000000000006
0.449 0.167 0.28200000000000003
0.363 0.223 0.13999999999999999
0.344 0.17 0.17399999999999996
0.161 0.217 -0.055999999999999994
0.569 0.255 0.31399999999999995
0.356 0.315 0.04099999999999998
0.391 0.294 0.09700000000000003
0.197 0.283 -0.08599999999999997
0.572 0.222 0.35
0.455 0.43 0.025000000000000022
0.43 0.551 -0.12100000000000005
0.477 0.476 0.0010000000000000009
0.801 0.301 0.5
0.826 0.169 0.6569999999999999
0.794 0.58 0.21400000000000008
0.933 0.667 0.266
0.878 0.403 0.475
0.894 0.247 0.647
0.715 0.498 0.21699999999999997
0.866 0.65 0.21599999999999997
0.783 0.346 0.43700000000000006
0.724 0.541 0.182999999999

In [100]:
print(f"XSS ACC diff: {get_max_difference(pd.concat([df_rq1_xss_acc]))}")


0.803 0.767 0.03600000000000003
0.786 0.787 -0.0010000000000000009
0.897 0.953 -0.05599999999999994
0.926 0.873 0.05300000000000005
0.751 0.723 0.028000000000000025
0.702 0.812 -0.1100000000000001
0.839 0.831 0.008000000000000007
0.87 0.813 0.05700000000000005
0.796 0.784 0.01200000000000001
0.753 0.768 -0.015000000000000013
0.762 0.832 -0.06999999999999995
0.805 0.757 0.04800000000000004
0.614 0.54 0.07399999999999995
0.58 0.579 0.0010000000000000009
0.591 0.557 0.03399999999999992
0.569 0.594 -0.025000000000000022
0.656 0.562 0.09399999999999997
0.58 0.619 -0.039000000000000035
0.606 0.601 0.0050000000000000044
0.539 0.625 -0.08599999999999997
0.784 0.593 0.19100000000000006
0.672 0.701 -0.028999999999999915
0.695 0.628 0.06699999999999995
0.762 0.652 0.10999999999999999
0.744 0.599 0.14500000000000002
0.706 0.693 0.013000000000000012
0.694 0.759 -0.06500000000000006
0.685 0.721 -0.03599999999999992
0.813 0.604 0.20899999999999996
0.817 0.576 0.241
0.814 0.748 0.06599999999999995
0.9

In [101]:
print(f"XSS ACC diff: {get_max_difference(pd.concat([df_rq1_xss_acc, df_rq1_xss_f2]))}")


0.803 0.767 0.03600000000000003
0.786 0.787 -0.0010000000000000009
0.897 0.953 -0.05599999999999994
0.926 0.873 0.05300000000000005
0.751 0.723 0.028000000000000025
0.702 0.812 -0.1100000000000001
0.839 0.831 0.008000000000000007
0.87 0.813 0.05700000000000005
0.796 0.784 0.01200000000000001
0.753 0.768 -0.015000000000000013
0.762 0.832 -0.06999999999999995
0.805 0.757 0.04800000000000004
0.614 0.54 0.07399999999999995
0.58 0.579 0.0010000000000000009
0.591 0.557 0.03399999999999992
0.569 0.594 -0.025000000000000022
0.656 0.562 0.09399999999999997
0.58 0.619 -0.039000000000000035
0.606 0.601 0.0050000000000000044
0.539 0.625 -0.08599999999999997
0.784 0.593 0.19100000000000006
0.672 0.701 -0.028999999999999915
0.695 0.628 0.06699999999999995
0.762 0.652 0.10999999999999999
0.744 0.599 0.14500000000000002
0.706 0.693 0.013000000000000012
0.694 0.759 -0.06500000000000006
0.685 0.721 -0.03599999999999992
0.813 0.604 0.20899999999999996
0.817 0.576 0.241
0.814 0.748 0.06599999999999995
0.9

In [86]:
print(f"XSS ACC diff: {get_max_difference(df_rq1_xss_acc)}")
print(f"XSS F2 diff: {get_max_difference(df_rq1_xss_f2)}")
print(f"SQLi ACC diff: {get_max_difference(df_rq1_sqli_acc)}")
print(f"SQLi F2 diff: {get_max_difference(df_rq1_sqli_f2)}")
print(f"XSS diff: {get_max_difference(pd.concat([df_rq1_xss_acc, df_rq1_xss_f2]))}")
print(f"SQLi diff: {get_max_difference(pd.concat([df_rq1_sqli_acc, df_rq1_sqli_f2]))}")
print(f"ACC diff: {get_max_difference(pd.concat([df_rq1_xss_acc, df_rq1_sqli_acc]))}")
print(f"F2 diff: {get_max_difference(pd.concat([df_rq1_sqli_f2, df_rq1_xss_f2]))}")
print(f"AVG diff: {get_max_difference(df_rq1)}")

XSS ACC diff: 0.29200000000000004
XSS F2 diff: 0.6569999999999999
SQLi ACC diff: 0.22699999999999998
SQLi F2 diff: 0.669
XSS diff: 0.29200000000000004
SQLi diff: 0.22699999999999998
ACC diff: 0.29200000000000004
F2 diff: 0.669
AVG diff: 0.29200000000000004


RQ2

In [11]:
xss_acc_file_rq2 = "/workspaces/RAG_secure_code_generation/plots_rq2.csv"
xss_f2_file_rq2 = "/workspaces/RAG_secure_code_generation/plots_rq2_f2.csv"
sqli_acc_file_rq2 = "/workspaces/RAG_secure_code_generation/plots_rq2_sqli.csv"
sqli_f2_file_rq2 = "/workspaces/RAG_secure_code_generation/plots_rq2_sqli_f2.csv"

In [56]:
df_rq2_xss_acc = pd.read_csv(xss_acc_file_rq2)
df_rq2_xss_f2 = pd.read_csv(xss_f2_file_rq2)
df_rq2_sqli_acc= pd.read_csv(sqli_acc_file_rq2)
df_rq2_sqli_f2 = pd.read_csv(sqli_f2_file_rq2)

#rename all the columns ending with imrpvoement with the split [0:2]
df_rq2_xss_acc = df_rq2_xss_acc.rename(columns={col: '_'.join(col.split("_")[0:2])+"_improv" for col in df_rq2_xss_acc.columns if col.endswith("improvement")})
df_rq2_xss_f2 = df_rq2_xss_f2.rename(columns={col: '_'.join(col.split("_")[0:2])+"_improv" for col in df_rq2_xss_f2.columns if col.endswith("improvement")})
df_rq2_sqli_acc = df_rq2_sqli_acc.rename(columns={col: '_'.join(col.split("_")[0:2])+"_improv" for col in df_rq2_sqli_acc.columns if col.endswith("improvement")})
df_rq2_sqli_f2 = df_rq2_sqli_f2.rename(columns={col: '_'.join(col.split("_")[0:2])+"_improv" for col in df_rq2_sqli_f2.columns if col.endswith("improvement")})


df_rq2 = pd.concat([df_rq2_xss_acc,
                    df_rq2_xss_f2,
                    df_rq2_sqli_acc,
                    df_rq2_sqli_f2
                    ])


In [57]:
df_rq2_xss_acc.head()

,experiment,dataset,avg_accuracy,top_1_acc,top_1_acc_diff,top_1_improv,top_3_acc,top_3_acc_diff,top_3_improv,top_5_acc,top_5_acc_diff,top_5_improv
0,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_zero_shot,0.765,0.838,0.146,0.073,0.852,0.117,0.088,0.833,0.125,0.068
1,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_3,0.765,0.923,0.061,0.158,0.887,0.082,0.122,0.877,0.081,0.112
2,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_1,0.765,0.810,0.174,0.046,0.830,0.139,0.065,0.854,0.104,0.089
3,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_5,0.765,0.968,0.016,0.204,0.901,0.069,0.136,0.866,0.092,0.101
4,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_1.0_zero_shot_5,0.765,0.835,0.148,0.071,0.854,0.115,0.089,0.845,0.113,0.080


In [58]:
def get_avg_improvement(df):
    #keep only the columns starting with no_rag or rag
    df = df.filter(regex='improvement')


    return df.sum().mean()/len(df)

In [78]:
def get_max_improvement(df):
    #keep only the columns starting with no_rag or rag
    df = df.filter(regex='improvement')


    return df.max().mean()

In [59]:
len(df_rq2)

36064

In [60]:
df_rq2.head()

,experiment,dataset,avg_accuracy,top_1_acc,top_1_acc_diff,top_1_improv,top_3_acc,top_3_acc_diff,top_3_improv,top_5_acc,top_5_acc_diff,top_5_improv,avg_f2,top_1_f2,top_3_f2,top_5_f2
0,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_zero_shot,0.765,0.838,0.146,0.073,0.852,0.117,0.088,0.833,0.125,0.068,NaN,NaN,NaN,NaN
1,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_3,0.765,0.923,0.061,0.158,0.887,0.082,0.122,0.877,0.081,0.112,NaN,NaN,NaN,NaN
2,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_1,0.765,0.810,0.174,0.046,0.830,0.139,0.065,0.854,0.104,0.089,NaN,NaN,NaN,NaN
3,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_0.5_few_shot_5,0.765,0.968,0.016,0.204,0.901,0.069,0.136,0.866,0.092,0.101,NaN,NaN,NaN,NaN
4,gpt-4-0125-preview_0.5_rag_few_shot_3,anthropic-claude-3-haiku_1.0_zero_shot_5,0.765,0.835,0.148,0.071,0.854,0.115,0.089,0.845,0.113,0.080,NaN,NaN,NaN,NaN


In [61]:
len(df_rq2_xss_acc)+len(df_rq2_xss_f2) +len(df_rq2_sqli_acc)+len(df_rq2_sqli_f2)

36064

In [62]:
print(f"XSS ACC diff: {get_max_improvement(df_rq2_xss_acc)}")
print(f"XSS F2 diff: {get_max_improvement(df_rq2_xss_f2)}")
print(f"SQLi ACC diff: {get_max_improvement(df_rq2_sqli_acc)}")
print(f"SQLi F2 diff: {get_max_improvement(df_rq2_sqli_f2)}")
print(f"XSS diff: {get_max_improvement(pd.concat([df_rq2_xss_acc, df_rq2_xss_f2]))}")
print(f"SQLi diff: {get_max_improvement(pd.concat([df_rq2_sqli_acc, df_rq2_sqli_f2]))}")
print(f"Acc diff: {get_max_improvement(pd.concat([df_rq2_xss_acc, df_rq2_sqli_acc]))}")
print(f"F2 diff: {get_max_improvement(pd.concat([df_rq2_xss_f2, df_rq2_sqli_f2]))}")
print(f"AVG diff: {get_max_improvement(df_rq2)}")

XSS ACC diff: 0.13537796489563572
XSS F2 diff: 0.246287673940544
SQLi ACC diff: 0.10480802083333333
SQLi F2 diff: 0.042670868055555554
XSS diff: 0.19083281941808983
SQLi diff: 0.07373944444444444
Acc diff: 0.11910292812777283
F2 diff: 0.1378847604259095
AVG diff: 0.12849384427684116


RQ3

In [63]:
best_results_xss_file = "/workspaces/RAG_secure_code_generation/experiments/task_detect_xss_simple_prompt/template_create_function_readable/prompt_parameters_empty/best_experiment_xss.json"
best_results_sqli_file = "/workspaces/RAG_secure_code_generation/experiments/task_detect_sqli_extended/template_create_function_readable/prompt_parameters_empty/best_experiment_sqli.json"

In [64]:
cnn_results_file = "/workspaces/RAG_secure_code_generation/src/detection_models/runs/cnn/run_0/test_results.json"
mlp_results_file = "/workspaces/RAG_secure_code_generation/src/detection_models/runs/mlp/run_0/test_results.json"
sofia_results_file = "/workspaces/RAG_secure_code_generation/sofia.json"

In [65]:
with open(best_results_xss_file, 'r') as f:
    best_results_xss = json.load(f)
with open(best_results_sqli_file, 'r') as f:
    best_results_sqli = json.load(f)
with open(cnn_results_file, 'r') as f:
    cnn_results = json.load(f)
with open(mlp_results_file, 'r') as f:
    mlp_results = json.load(f)
with open(sofia_results_file, 'r') as f:
    sofia_results = json.load(f)

In [66]:
top_ks = [1,3,5]

In [67]:
xss_degradation_acc = 0
sqli_degradation_acc = 0
xss_degradation_f2 = 0
sqli_degradation_f2 = 0

for k in top_ks:
    temp_degr = cnn_results["accuracy"]/100 - best_results_xss["test_results_synth"]["top_k_metrics"][f"top_{k}_accuracy"]
    temp_degr += (mlp_results["accuracy"]/100 - best_results_xss["test_results_synth"]["top_k_metrics"][f"top_{k}_accuracy"])
    xss_degradation_acc += temp_degr/2
    temp_degr = cnn_results["f2score"] - best_results_xss["test_results_synth"]["top_k_metrics"][f"top_{k}_f2"]
    temp_degr += (mlp_results["f2score"] - best_results_xss["test_results_synth"]["top_k_metrics"][f"top_{k}_f2"])
    xss_degradation_f2 += temp_degr/2
    sqli_degradation_acc += sofia_results["acc"] - best_results_sqli["test_results_synth"]["top_k_metrics"][f"top_{k}_accuracy"]
    sqli_degradation_f2 += sofia_results["f2"] - best_results_sqli["test_results_synth"]["top_k_metrics"][f"top_{k}_f2"]


xss_degradation_acc /= len(top_ks)
sqli_degradation_acc /= len(top_ks)
xss_degradation_f2 /= len(top_ks)
sqli_degradation_f2 /= len(top_ks)


In [68]:
print(f"XSS ACC diff: {xss_degradation_acc}")
print(f"XSS F2 diff: {xss_degradation_f2}")
print(f"SQLi ACC diff: {sqli_degradation_acc}")
print(f"SQLi F2 diff: {sqli_degradation_f2}")
print(f"XSS diff: {(xss_degradation_acc + xss_degradation_f2)/2}")
print(f"SQLi diff: {(sqli_degradation_acc + sqli_degradation_f2)/2}")
print(f"Acc diff: {(xss_degradation_acc + sqli_degradation_acc)/2}")
print(f"F2 diff: {(sqli_degradation_acc + sqli_degradation_acc)/2}")
print(f"AVG diff: {(xss_degradation_acc + xss_degradation_f2 + sqli_degradation_acc + sqli_degradation_f2)/5}")

XSS ACC diff: 0.009619827767275787
XSS F2 diff: 0.018568990363301324
SQLi ACC diff: 0.022753003494066986
SQLi F2 diff: 0.025216136066335066
XSS diff: 0.014094409065288555
SQLi diff: 0.023984569780201026
Acc diff: 0.016186415630671386
F2 diff: 0.022753003494066986
AVG diff: 0.015231591538195833


RQ4

In [69]:
rq4_file = "/workspaces/RAG_secure_code_generation/rq4.json"

In [70]:
with open(rq4_file, 'r') as f:
    rq4_results = json.load(f)

In [71]:
xss_loss_acc = 0
sqli_loss_acc = 0
xss_loss_f2 = 0
sqli_loss_f2 = 0
for k,v in rq4_results.items():
    optimal_xss_acc = v["xss_to_sqli"]["original_acc"]
    optimal_xss_f2 = v["xss_to_sqli"]["original_f2"]
    optimal_sqli_acc = v["sqli_to_xss"]["original_acc"]
    optimal_sqli_f2 = v["sqli_to_xss"]["original_f2"]  

    new_xss_acc = v["sqli_to_xss"]["transfer_acc"]
    new_xss_f2 = v["sqli_to_xss"]["transfer_f2"]
    new_sqli_acc = v["xss_to_sqli"]["transfer_acc"]
    new_sqli_f2 = v["xss_to_sqli"]["transfer_f2"]  

    xss_loss_acc += optimal_xss_acc - new_xss_acc
    xss_loss_f2 += optimal_xss_f2 - new_xss_f2
    sqli_loss_acc += optimal_sqli_acc - new_sqli_acc
    sqli_loss_f2 += optimal_sqli_f2 - new_sqli_f2

xss_loss_acc /= len(rq4_results.keys())
xss_loss_f2 /= len(rq4_results.keys())
sqli_loss_acc /= len(rq4_results.keys())
sqli_loss_f2 /= len(rq4_results.keys())


In [72]:
print(f"XSS ACC diff: {xss_loss_acc}")
print(f"XSS F2 diff: {xss_loss_f2}")
print(f"SQLi ACC diff: {sqli_loss_acc}")
print(f"SQLi F2 diff: {sqli_loss_f2}")
print(f"XSS diff: {(xss_loss_acc + xss_loss_f2)/2}")
print(f"SQLi diff: {(sqli_loss_acc + sqli_loss_f2)/2}")
print(f"Acc diff: {(xss_loss_acc + sqli_loss_acc)/2}")
print(f"F2 diff: {(xss_loss_f2 + sqli_loss_f2)/2}")
print(f"AVG diff: {(xss_loss_acc + xss_loss_f2 + sqli_loss_acc + sqli_loss_f2)/5}")

XSS ACC diff: 0.0930350766645663
XSS F2 diff: 0.02355123724999168
SQLi ACC diff: 0.07770226808736753
SQLi F2 diff: 0.11244413108866969
XSS diff: 0.05829315695727899
SQLi diff: 0.0950731995880186
Acc diff: 0.08536867237596692
F2 diff: 0.06799768416933068
AVG diff: 0.06134654261811904
